In [1]:
#Imports
import numpy as np
import random
import pandas as pd
from ACO import *
from scipy.spatial import distance

# Ant Colony Optimisation Algorithm

## <center> Probability matrix </center>

Probability between vertices is given by:
\begin{equation*} P_{ij} = \dfrac{(\tau_{ij})^\alpha (n(c_{ij}))^\beta}{\sum_{j=0}^{m} (\tau_{ij})^\alpha (n(c_{ij}))^\beta }  \end{equation*}


where $c_{ij}$ is the length of the edge 

$\tau_{ij}$ is the pheromone count on the edge $c_{ij}$

$n(c_{ij}) = \dfrac{1}{c_{ij}}$

can be interpreted as the Probability of going to node j from i

In [4]:
def updateProbMatrix(A,P,alpha,beta):
    """
        Generates the probability matrix based on the pheromone and distance matrix

        Inputs
            A     : the distance matrix
            P     : the Pheromone Matrix
            alpha : the Pheromone weight value
            beta  : the distance weight value

        Outputs
            X     : the probability matrix
    """

    n = len(A)

    X = np.zeros((n,n))

    for i in range(0,n,1):

        #calculating the total weight across path
        deno = 0

        for j in range(0,n,1):

            if(A[i,j]!=0):
                deno += (P[i,j]**alpha)*((1/A[i,j])**beta)

        #updating probability matrix
        for j in range(0,n,1):
            
            if i!=j and A[i,j]!=0:
                X[i,j] = (P[i,j]**alpha)*((1/A[i,j])**beta)/deno
            else:
                X[i,j] = 0
    
    return X

In [5]:
#TESTING#
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = updateProbMatrix(A,P,alpha,beta)
Prob

array([[0.        , 0.28368794, 0.70921986, 0.0070922 ],
       [0.43478261, 0.        , 0.02173913, 0.54347826],
       [0.59288538, 0.01185771, 0.        , 0.39525692],
       [0.00849858, 0.42492918, 0.56657224, 0.        ]])

## <center> Accumulator</center>

generates the Accumulator vector. which is basically the sum of itself and all the probabilities to the right of it

for example given the probabilities $v = [0.76,0.19,0.05]$ 

the Accumulative vector will be $u = [1,0.24,0.05]$

after this a random number will be generated. $r \in [0,1]$

then the function will return the index in which r is inbetween the vector u

for example if $ 0.24 \leq r \leq 1 $ the function will return 0

In [6]:
def accumulator(v):
    """
        Creates the accumulator vector from the input given
    """

    u = []

    for i in range(len(v)):
        temp = sum(v[i:])
        u.append(temp)

    r = random.uniform(0,u[0])
    index = -1

    for i in range(0,len(u),1):
        
        if i!=len(u)-1:
            if (r<=u[i]) and (r>u[i+1]):
                index = i
                break
        else:
            if (r>0) and (r<=u[-1]):
                index = i

    return index

In [7]:
#TESTING
#v = [0.76,0.19,0.05]
v = [0,0.28368794,0.70921986,0.0070922]
accumulator(v)

2

In [8]:
def ant_path(A,Prob):
    """
        Calculates the path the ant with take

        Inputs:
            A    : the Distance matrix
            Prob : the probability matrix

        output:
            C    : a matrix with 1/d at the edges the ant passed. where d is the distance traveled 
    """

    n = len(A)
    P = np.copy(Prob)
    C = np.zeros((n,n))
    distance = 0
    path = []
    i = 0
    count = 0

    while count<n:

        v = P[i,:]
        
        #reducing the probability of going to start#
        if i!=0 and count<n-1:
            v[0] = np.min(v[np.nonzero(v)])/n

        j = accumulator(v)
        distance+= A[i][j]
        path.append([i,j])

        #making sure ant doesnt visit same node twice
        if i!=0:
            P[:,i] = 0

        i = j
        count+=1

    for x in range(0,len(path),1):
        [i,j] = path[x]
        C[i,j] += 1/distance   
    return C     

In [9]:
#TESTING#
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
P = np.ones((4,4))
alpha = 1
beta = 1
Prob = updateProbMatrix(A,P,alpha,beta)
ant_path(A,Prob)

array([[0.        , 0.        , 0.00325733, 0.00325733],
       [0.00325733, 0.        , 0.        , 0.        ],
       [0.        , 0.00325733, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [10]:
def final_walk(A,Prob):
    """
        Calculates the path the last ant with take

        Inputs:
            A    : the Distance matrix
            Prob : the probability matrix

        output:
            C    : a matrix with 1/d at the edges the ant passed. where d is the distance traveled 
    """

    n = len(A)
    P = np.copy(Prob)
    C = np.zeros((n,n))
    distance = 0
    path = []
    i = 0
    count = 0

    while count<n:

        v = P[i,:]
        
        #reducing the probability of going to start#
        if i!=0 and count<n-1:
            v[0] = np.min(v[np.nonzero(v)])/n

        j = accumulator(v)
        distance+= A[i][j]

        if j!=-1:
            path.append([i,j])

        if j == 0:
            break
        #making sure ant doesnt visit same node twice
        if i!=0:
            P[:,i] = 0

        # P[:,i] = 0
        i = j
        count+=1

    return distance,path

In [11]:
def ACO(A,p,alpha,beta,n,k):
    """
    Finds optimal route using Ant Colony Optimisation techniques
    
    Inputs:
        A: Distance Matrix
        p: (scalar) evaporation rate
        alpha: (scalar) parameter that affects pheromone weighting
        beta: (scalar) parameter that affects distance weighting
        n: (scalar) number of interations to be performed
        k: (scalar) number of ants to be used
        
    Output:
        path: set of 2-tuples of route to be taken
    """

    #Pheromone matrix#
    X = np.ones((len(A),len(A)))

    for i in range(0,n,1):

        #pheromone update matrix#
        C = np.zeros((len(A),len(A)))

        #probability matrix#
        P = updateProbMatrix(A,X,alpha,beta)

        #constructing ant paths#
        for j in range(0,k,1):
            C+= ant_path(A,P)
        
        #updating pheromone#
        X = (1-p)*X + C
    
    dist,path = final_walk(A,P)

    while path[-1][1]!= 0 or len(path)!=len(A):
        dist,path = final_walk(A,P)

    print('\n============================================\n')
    print('path taken \n')
    print(path)
    print('\npath distance was:',dist)
    print('\n============================================\n')

### Test 1

In [12]:
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
p = 0.5
alpha = 1
beta = 4
n = 4
k = 3
ACO(A,p,alpha,beta,n,k)



path taken 

[[0, 2], [2, 3], [3, 1], [1, 0]]

path distance was: 14




In [14]:
A = np.array([[0,5,2,200],[5,0,100,4],[2,100,0,3],[200,4,3,0]])
p = 0.5
alpha = 1
beta = 4
n = 4
k = 3
model = ACO()
model.ACO(A,p,alpha,beta,n,k)

TypeError: ACO() missing 6 required positional arguments: 'A', 'p', 'alpha', 'beta', 'n', and 'k'

### Test 2

In [22]:
data = pd.read_csv('Data/br17.csv',delim_whitespace=True)
X = data.to_numpy()
#formating data#
for i  in range(len(X)):
    X[i][i] = 0
print(X)

[[ 0  3  5 48 48  8  8  5  5  3  3  0  3  5  8  8  5]
 [ 3  0  3 48 48  8  8  5  5  0  0  3  0  3  8  8  5]
 [ 5  3  0 72 72 48 48 24 24  3  3  5  3  0 48 48 24]
 [48 48 74  0  0  6  6 12 12 48 48 48 48 74  6  6 12]
 [48 48 74  0  0  6  6 12 12 48 48 48 48 74  6  6 12]
 [ 8  8 50  6  6  0  0  8  8  8  8  8  8 50  0  0  8]
 [ 8  8 50  6  6  0  0  8  8  8  8  8  8 50  0  0  8]
 [ 5  5 26 12 12  8  8  0  0  5  5  5  5 26  8  8  0]
 [ 5  5 26 12 12  8  8  0  0  5  5  5  5 26  8  8  0]
 [ 3  0  3 48 48  8  8  5  5  0  0  3  0  3  8  8  5]
 [ 3  0  3 48 48  8  8  5  5  0  0  3  0  3  8  8  5]
 [ 0  3  5 48 48  8  8  5  5  3  3  0  3  5  8  8  5]
 [ 3  0  3 48 48  8  8  5  5  0  0  3  0  3  8  8  5]
 [ 5  3  0 72 72 48 48 24 24  3  3  5  3  0 48 48 24]
 [ 8  8 50  6  6  0  0  8  8  8  8  8  8 50  0  0  8]
 [ 8  8 50  6  6  0  0  8  8  8  8  8  8 50  0  0  8]
 [ 5  5 26 12 12  8  8  0  0  5  5  5  5 26  8  8  0]]


In [25]:
#p = 0.3;alpha=30;beta=40;n=4;k=3#
A = X
p = 0.3
alpha = 10
beta = 5
n = 100
k = 7
ACO(A,p,alpha,beta,n,k)



path taken 

[[0, 9], [9, 11], [11, 10], [10, 13], [13, 1], [1, 7], [7, 6], [6, 3], [3, 15], [15, 8], [8, 5], [5, 4], [4, 14], [14, 16], [16, 12], [12, 2], [2, 0]]

path distance was: 89


